In [11]:
import csv
import cv2
import pytesseract 
from pytesseract import image_to_string
import pandas as pd
import re
import os
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
%run helper-extractingVideos.ipynb

In [12]:
def extract_from_video(filePath,videoName,template,templateRes):
        # Open the video using OpenCV
    
    desired_width = 1631
    desired_height = 907
    cap = cv2.VideoCapture(filePath)
    #,apiPreference=cv2.CAP_ANY,params=[
    #cv2.CAP_PROP_FRAME_WIDTH,desired_width ,
    #cv2.CAP_PROP_FRAME_HEIGHT, desired_height])
    
    #cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
    #cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)
    fps=cap.get(cv2.CAP_PROP_FPS)
    print("FPS:",fps)
    minToSkip=11
    framesToSkip=fps*60*minToSkip
    framesToSkipEverySec=fps-1
    currentFrame=0
    currentFrameInSec=0
    jumpStarted=False
    sno=1
    row={}#changed

    # Initialize a list to store the scores

    print("start")

    oldText="old"
    # Process each frame of the video
    while True:
        # Read the next frame
        ret, frame = cap.read()
        currentFrame+=1
        currentFrameInSec+=1
        if currentFrame<framesToSkip :
            if currentFrame%(fps*60)==0:
                print(currentFrame//(fps*60) ," min(s) Skipped")
            continue
        if currentFrame//(fps*60)<minToSkip:
            continue
        if currentFrameInSec<framesToSkipEverySec:
            continue



        # Break the loop if the video has ended
        if not ret:
            print("completed")
            break

        current_pos = cap.get(cv2.CAP_PROP_POS_MSEC)
        total_seconds = int(current_pos // 1000)
        minutes = total_seconds // 60
        seconds = total_seconds % 60
        timeInVideo =f"{minutes} minutes and {seconds} seconds"
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

       
    
        
        if not jumpStarted:
            cropped=extract_frame(frame,template)
            print("Cropped :" , cropped)
            if cropped:
                jumpStarted=True
            #plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
                row['Time'] = timeInVideo
                print(timeInVideo," jump started")
                if 'Time' in row:  # Check if 'Time' key exists in row
                # Extract minutes and seconds from time duration string
                    time_str = row['Time']
                    minutes_str, seconds_str = re.match(r'(\d+) minutes and (\d+) seconds', time_str).groups()
                # Convert minutes and seconds to integers
                    minutes = int(minutes_str)
                    seconds = int(seconds_str)
                # Calculate total seconds
                    totalSeconds = minutes * 60 + seconds            

      
        if jumpStarted:
            cropped=extract_frame(frame,templateRes)
            if cropped:
                jumpStarted = False
                print(timeInVideo," jump scores announced")
                #plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
                extract_clip(filePath,'ExtractedVideos/'+str(sno)+'_'+videoName+'.mp4',totalSeconds,int(total_seconds))
                sno+=1
                #print(text)
                print(row)
                #save_data(row,"scores")

        

        currentFrameInSec=0    
    cap.release()

In [13]:
templateLondon = cv2.imread('helperImages/template1.jpg', cv2.IMREAD_GRAYSCALE)
templateResLondon = cv2.imread('helperImages/template2.png', cv2.IMREAD_GRAYSCALE)
extract_from_video("video/Women's_High_Jump_Final_Tokyo_Replays.mp4","Women's_High_Jump_Final_Tokyo_Replays",templateLondon,templateResLondon)

FPS: 29.97002997002997
start
Length of loc[0]: 0
Length of loc[1]: 0
Cropped : False
Length of loc[0]: 445
Length of loc[1]: 445
Cropped : True
11 minutes and 0 seconds  jump started
Length of loc[0]: 6992
Length of loc[1]: 6992
11 minutes and 0 seconds  jump scores announced


error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)
